<a href="https://colab.research.google.com/github/asigalov61/Quintessential-Viterbi/blob/main/Quintessential_Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quintessential Viterbi (v. 1.3)

***

## Full-featured Viterbi MIDI Music Augmentator

***

#### Project Los Angeles

#### Tegridy Code 2020

# Setup Environment

In [ ]:
#@title Install dependencies
print('Installing dependencies...')
print('Please stand-by...')
!git clone https://github.com/asigalov61/Quintessential-Viterbi
%cd /content/Quintessential-Viterbi

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth mido #pretty_midi

print('Installing FluidSynth...')
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Done! Enjoy! :)')

In [ ]:
#@title Import Modules
%cd /content/Quintessential-Viterbi
import constants

import pretty_midi
import midi_io, midi_synth
import sequences_lib
import melodies_lib, melody_inference
import chords_lib, chord_inference
import notebook_utils

import copy

from google.colab import files
import IPython

# Generate Agumented Viterbi Music from your MIDI

In [ ]:
#@title Please note that Viterbi algorithm may throw errors occasionally so please try generate again if that happens. Also, remove_drums code is a bit buggy so switch this option if you keep getting errors. Same applies for other settings and/or division by zero cases.
full_path_to_MIDI_file = "/content/Quintessential-Viterbi/Sample_MIDI.mid" #@param {type:"string"}
ignore_polyphonic_notes = True #@param {type:"boolean"}
MIDI_instrument_number_for_output_melody_notes = 40 #@param {type:"slider", min:0, max:127, step:1}
MIDI_program_for_output_melody_notes = 0 #@param {type:"slider", min:0, max:127, step:1}
start_output_melody_at_this_second = 0 #@param {type:"slider", min:0, max:200, step:0.1}
squash_melody = True #@param {type:"boolean"}
minimum_squash_pitch = 40 #@param {type:"slider", min:0, max:127, step:1}
maximum_squash_pitch = 60 #@param {type:"slider", min:0, max:127, step:1}
transpose_squashed_melody_to_key = 0 #@param ["None", "0", "1", "2", "3", "4", "5", "6", "7", "None"] {type:"raw"}
transpose_melody_velocity = 10 #@param {type:"slider", min:-30, max:30, step:1}
quarter_notes_per_minute = 120 #@param {type:"slider", min:1, max:480, step:1}
chords_inference_type = "Viterbi" #@param ["Viterbi", "Notewise"]
infer_chords_from = "Original Composition"
notewise_chords_inference_min_notes_per_chord = 3
number_of_chords_per_bar = None #@param ["None", "1", "2", "4", "8", "16", "32"] {type:"raw"}
chord_key_change_probability = 0.95 #@param {type:"slider", min:0.01, max:2, step:0.01}
chord_change_probability = 0.95 #@param {type:"slider", min:0.01, max:2, step:0.01}
chord_pitch_out_of_key_probability = 0.94 #@param {type:"slider", min:0.01, max:2, step:0.01}
melody_notes_per_chord = 1 #@param {type:"slider", min:1, max:20, step:1}
add_key_signature = True #@param {type:"boolean"}
chord_MIDI_instrument_number = 0 #@param {type:"slider", min:0, max:127, step:1}
chord_MIDI_program_number = 0 #@param {type:"slider", min:0, max:127, step:1}
chords_main_octave = 5 #@param {type:"slider", min:1, max:10, step:1}
chords_bass_octave = 4 #@param {type:"slider", min:1, max:10, step:1}
transpose_chords_velocity = 10 #@param {type:"slider", min:-30, max:30, step:1}
augmentation_min_composition_stretch_ratio = -0.25 #@param {type:"slider", min:-2, max:2, step:0.25}
augmentation_max_composition_stretch_ratio = 0.25 #@param {type:"slider", min:-2, max:2, step:0.25}
augmentation_min_composition_transpose_number = -10 #@param {type:"slider", min:-30, max:30, step:1}
augmentation_max_composition_transpose_number = 10 #@param {type:"slider", min:-30, max:30, step:1}
augmentation_min_allowed_pitch = 30 #@param {type:"slider", min:0, max:127, step:1}
augmentation_max_allowed_pitch = 90 #@param {type:"slider", min:1, max:127, step:1}
augmentation_delete_out_of_range_notes = True #@param {type:"boolean"}
do_not_render_composition_to_audio = True #@param {type:"boolean"}
composition_quantization_steps_per_quarter = 16 #@param {type:"slider", min:1, max:32, step:1}

# Loading a MIDI file
melody_ns = midi_io.midi_file_to_note_sequence(full_path_to_MIDI_file)
melody_direct = melodies_lib.midi_file_to_melody(full_path_to_MIDI_file, steps_per_quarter=composition_quantization_steps_per_quarter, qpm=None, ignore_polyphonic_notes=ignore_polyphonic_notes)

# Augmenting the original composition
augmented_sequence = sequences_lib.augment_note_sequence(
        melody_ns,
        min_stretch_factor=augmentation_min_composition_stretch_ratio,
        max_stretch_factor=augmentation_max_composition_stretch_ratio,
        min_transpose=augmentation_min_composition_transpose_number,
        max_transpose=augmentation_max_composition_transpose_number,
        min_allowed_pitch=augmentation_min_allowed_pitch,
        max_allowed_pitch=augmentation_max_allowed_pitch,
        delete_out_of_range_notes=augmentation_delete_out_of_range_notes)

mel = sequences_lib.remove_redundant_data(augmented_sequence)

# Generating and working with Melody/Notes

melody_ns1 = sequences_lib.quantize_note_sequence(mel, steps_per_quarter=composition_quantization_steps_per_quarter)

mel0 = melody_direct
if squash_melody:
  mel0.squash(min_note=minimum_squash_pitch, max_note=maximum_squash_pitch, transpose_to_key=transpose_squashed_melody_to_key)

mel1 = mel0.to_sequence(velocity=transpose_melody_velocity, 
                instrument=MIDI_instrument_number_for_output_melody_notes, 
                program=MIDI_program_for_output_melody_notes, 
                sequence_start_time=start_output_melody_at_this_second, 
                qpm=quarter_notes_per_minute
                )

mel = sequences_lib.remove_redundant_data(mel1)

# Generating and working with Accompaniment/Chords

if chords_inference_type == 'Viterbi':
  mel = sequences_lib.quantize_note_sequence(mel, steps_per_quarter=composition_quantization_steps_per_quarter)
  chord_inference.infer_chords_for_sequence(mel,
                                chords_per_bar=number_of_chords_per_bar,
                                key_change_prob=chord_key_change_probability,
                                chord_change_prob=chord_change_probability,
                                chord_pitch_out_of_key_prob=chord_pitch_out_of_key_probability,
                                chord_note_concentration=melody_notes_per_chord,
                                add_key_signatures=add_key_signature
                                )

else:
  sequences_lib.infer_dense_chords_for_sequence(mel, min_notes_per_chord=0)

mel_f1 = sequences_lib.quantize_note_sequence(mel, steps_per_quarter=composition_quantization_steps_per_quarter)

# Rendering the chords
chords_lib.BasicChordRenderer(velocity=transpose_chords_velocity,
                                  instrument=chord_MIDI_instrument_number, 
                                  program=chord_MIDI_program_number, 
                                  octave=chords_main_octave, 
                                  bass_octave=chords_bass_octave).render(mel_f1)                               

mel2 = sequences_lib.remove_redundant_data(mel_f1)

# Done! Creating output, rendering audio, and crunching output stats.
print('Original MIDI composition:')
melody_o = midi_io.midi_file_to_note_sequence(full_path_to_MIDI_file)
notebook_utils.plot_sequence(melody_o)
print('Augmented Viterbi MIDI composition:')
notebook_utils.plot_sequence(mel2)
print('Some stats:')
print('Note Histogram:', melodies_lib.Melody.get_note_histogram(mel0))
print('Major Key Histogram:', melodies_lib.Melody.get_major_key_histogram(mel0))
print('Major Key:', melodies_lib.Melody.get_major_key(mel0))
if not do_not_render_composition_to_audio:
  print('Synthesizing the output Viterbi MIDI. Please stand-by... ')
  synth=midi_synth.fluidsynth(mel2, sample_rate=32000, sf2_path='/usr/share/sounds/sf2/FluidR3_GM.sf2')
  notebook_utils.colab_play(synth, sample_rate=32000, autoplay=True)
print('Downloading Viterbi MIDI composition... ')
print('Task complete! Enjoy! :)')

#midi_io.sequence_proto_to_midi_file(melody_ns, 'Original_Melody.mid')
midi_io.sequence_proto_to_midi_file(mel2, 'Viterbi_Melody.mid')

#files.download('Original_Melody.mid')
files.download('Viterbi_Melody.mid')